In [109]:
import numpy as np
import pandas as pd
from typing import Union
from tqdm.notebook import tqdm

class WRMSSEEvaluator(object):
    '''
    you can calculate public lb score like bellow.
    evaluator = WRMSSEEvaluator(train, valid, calendar, prices)
    _, scores = evaluator.score(valid_preds)
    calc_lb(preds_path, evaluator, sample_submission, train_end_date=1941)
    '''

    def __init__(self, train_df: pd.DataFrame, valid_df: pd.DataFrame, calendar: pd.DataFrame, prices: pd.DataFrame):
        train_y = train_df.loc[:, train_df.columns.str.startswith('d_')]
        train_target_columns = train_y.columns.tolist()
        weight_columns = train_y.iloc[:, -28:].columns.tolist()

        train_df['all_id'] = 0  # for lv1 aggregation

        id_columns = train_df.loc[:, ~train_df.columns.str.startswith('d_')].columns.tolist()
        valid_target_columns = valid_df.loc[:, valid_df.columns.str.startswith('d_')].columns.tolist()

        if not all([c in valid_df.columns for c in id_columns]):
            valid_df = pd.concat([train_df[id_columns], valid_df], axis=1, sort=False)

        self.train_df = train_df
        self.valid_df = valid_df
        self.calendar = calendar
        self.prices = prices

        self.weight_columns = weight_columns
        self.id_columns = id_columns
        self.valid_target_columns = valid_target_columns

        weight_df = self.get_weight_df()

        self.group_ids = (
            'all_id',
            'cat_id',
            'state_id',
            'dept_id',
            'store_id',
            'item_id',
            ['state_id', 'cat_id'],
            ['state_id', 'dept_id'],
            ['store_id', 'cat_id'],
            ['store_id', 'dept_id'],
            ['item_id', 'state_id'],
            ['item_id', 'store_id']
        )

        for i, group_id in enumerate(tqdm(self.group_ids)):
            train_y = train_df.groupby(group_id)[train_target_columns].sum()
            scale = []
            for _, row in train_y.iterrows():
                series = row.values[np.argmax(row.values != 0):]
                scale.append(((series[1:] - series[:-1]) ** 2).mean())
            setattr(self, f'lv{i + 1}_scale', np.array(scale))
            setattr(self, f'lv{i + 1}_train_df', train_y)
            setattr(self, f'lv{i + 1}_valid_df', valid_df.groupby(group_id)[valid_target_columns].sum())

            lv_weight = weight_df.groupby(group_id)[weight_columns].sum().sum(axis=1)
            setattr(self, f'lv{i + 1}_weight', lv_weight / lv_weight.sum())

    def get_weight_df(self) -> pd.DataFrame:
        day_to_week = self.calendar.set_index('d')['wm_yr_wk'].to_dict()
        weight_df = self.train_df[['item_id', 'store_id'] + self.weight_columns].set_index(['item_id', 'store_id'])
        weight_df = weight_df.stack().reset_index().rename(columns={'level_2': 'd', 0: 'value'})
        weight_df['wm_yr_wk'] = weight_df['d'].map(day_to_week)

        weight_df = weight_df.merge(self.prices, how='left', on=['item_id', 'store_id', 'wm_yr_wk'])
        weight_df['value'] = weight_df['value'] * weight_df['sell_price']
        weight_df = weight_df.set_index(['item_id', 'store_id', 'd']).unstack(level=2)['value']
        weight_df = weight_df.loc[zip(self.train_df.item_id, self.train_df.store_id), :].reset_index(drop=True)
        weight_df = pd.concat([self.train_df[self.id_columns], weight_df], axis=1, sort=False)
        return weight_df

    def rmsse(self, valid_preds: pd.DataFrame, lv: int) -> pd.Series:
        valid_y = getattr(self, f'lv{lv}_valid_df')
        score = ((valid_y - valid_preds) ** 2).mean(axis=1)
        scale = getattr(self, f'lv{lv}_scale')
        return (score / scale).map(np.sqrt)

    def score(self, valid_preds):
        assert self.valid_df[self.valid_target_columns].shape == valid_preds.shape

        if isinstance(valid_preds, np.ndarray):
            valid_preds = pd.DataFrame(valid_preds, columns=self.valid_target_columns)

        valid_preds = pd.concat([self.valid_df[self.id_columns], valid_preds], axis=1, sort=False)

        group_ids = []
        all_scores = []
        for i, group_id in enumerate(self.group_ids):
            lv_scores = self.rmsse(valid_preds.groupby(group_id)[self.valid_target_columns].sum(), i + 1)
            weight = getattr(self, f'lv{i + 1}_weight')
            lv_scores = pd.concat([weight, lv_scores], axis=1, sort=False).prod(axis=1)
            group_ids.append(group_id)
            all_scores.append(lv_scores.sum())

        return group_ids, all_scores


#train_end_dateはwrmsseに計算に使用する最新の日付
def calc_lb(preds_path, evaluator, sample_submission, train_end_date=1941) -> int:
    preds_valid = pd.read_csv(preds_path)
    ##以下追加したので注意。validation行とevaluation行があるから、30490行と60980行のどちらかで判断。
    preds_valid = preds_valid.iloc[30490:, :]
    preds_valid = preds_valid[preds_valid.id.str.contains("evaluation")]
    #preds_valid = preds_valid[preds_valid.id.str.contains("evaluation")]
    preds_valid = preds_valid.merge(sample_submission[["id", "order"]], on = "id").sort_values("order").drop(["id", "order"], axis = 1).reset_index(drop = True)
    preds_valid.rename(columns = {f"F{i}": f"d_{i+train_end_date-28}" for i in range(1, 29)}, inplace = True)

    _, scores = evaluator.score(preds_valid)
    score_public_lb = np.mean(scores)
    
    return score_public_lb

In [ ]:
def calc_lb_df(preds_df, evaluator, sample_submission, train_end_date=1941) -> int:
    preds_valid = preds_df
    ##以下追加したので注意。validation行とevaluation行があるから、30490行と60980行のどちらかで判断。
    preds_valid = preds_valid.iloc[30490:, :]
    preds_valid = preds_valid[preds_valid.id.str.contains("evaluation")]
    #preds_valid = preds_valid[preds_valid.id.str.contains("evaluation")]
    preds_valid = preds_valid.merge(sample_submission[["id", "order"]], on = "id").sort_values("order").drop(["id", "order"], axis = 1).reset_index(drop = True)
    preds_valid.rename(columns = {f"F{i}": f"d_{i+train_end_date-28}" for i in range(1, 29)}, inplace = True)

    _, scores = evaluator.score(preds_valid)
    score_public_lb = np.mean(scores)
    
    return score_public_lb

In [110]:
df = pd.read_csv('../data/input/m5-forecasting-accuracy/sales_train_evaluation.csv')
train = df.iloc[:, :-28]
valid = df.iloc[:, -28:]
# train = df.iloc[:, :-56]
# valid = df.iloc[:, -56:-28]
calendar = pd.read_csv('../data/input/m5-forecasting-accuracy/calendar.csv')
prices = pd.read_csv('../data/input/m5-forecasting-accuracy/sell_prices.csv')
sample_submission = pd.read_csv('../data/input/m5-forecasting-accuracy/sample_submission.csv')
sample_submission["order"] = range(sample_submission.shape[0])

In [111]:
evaluator = WRMSSEEvaluator(train, valid, calendar, prices)

In [174]:
# preds_path = '../data/output/submission_state_magic.csv'
preds_path = '../data/output/submission_magic_local'
# preds_path = '../data/output/categorical-emb.csv'

#結果が合わない時はvalidationがあっていないか、train_end_dateが合っていないかの可能性が高い。
calc_lb(preds_path, evaluator, sample_submission, train_end_date=1941)
# calc_lb(preds_path, evaluator, sample_submission, train_end_date=1941 - 28)

0.4739295997828486

In [113]:
preds_df = pd.read_csv(preds_path)
preds_df.head()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0.16592,0.15146,0.14512,0.15566,0.17568,0.20540,0.23400,0.16900,0.16620,...,0.18240,0.19952,0.20260,0.17050,0.16328,0.16328,0.16406,0.18526,0.21900,0.23600
1,HOBBIES_1_002_CA_1_validation,0.04060,0.03884,0.04996,0.04192,0.04316,0.04324,0.04844,0.03312,0.03610,...,0.04180,0.05048,0.06280,0.03938,0.03838,0.03870,0.03934,0.04146,0.05048,0.05346
2,HOBBIES_1_003_CA_1_validation,0.09356,0.08706,0.08746,0.08086,0.10830,0.10732,0.13028,0.06904,0.06552,...,0.09672,0.13180,0.16426,0.09126,0.08614,0.08204,0.07886,0.09086,0.13480,0.14512
3,HOBBIES_1_004_CA_1_validation,0.44060,0.41600,0.36940,0.37040,0.44640,0.51180,0.51600,0.40860,0.40080,...,0.48760,0.54920,0.58320,0.46640,0.44540,0.43760,0.45320,0.51720,0.63900,0.58640
4,HOBBIES_1_005_CA_1_validation,0.20040,0.19240,0.20000,0.22080,0.24380,0.27860,0.36440,0.24880,0.25260,...,0.27680,0.35120,0.36640,0.26520,0.24540,0.24020,0.24820,0.28160,0.34880,0.37060
